<a href="https://colab.research.google.com/github/manpreetlotay99/openai/blob/main/clipsegmodelipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00


In [ ]:
from transformers import CLIPSegConfig, CLIPSegModel, CLIPSegTextConfig, CLIPSegVisionConfig

In [ ]:

# Initializing a CLIPSegConfig with CIDAS/clipseg-rd64 style configuration
configuration = CLIPSegConfig()

# Initializing a CLIPSegModel (with random weights) from the CIDAS/clipseg-rd64 style configuration
model = CLIPSegModel(configuration)

# Accessing the model configuration
configuration = model.config

# We can also initialize a CLIPSegConfig from a CLIPSegTextConfig and a CLIPSegVisionConfig

# Initializing a CLIPSegText and CLIPSegVision configuration
config_text = CLIPSegTextConfig()
config_vision = CLIPSegVisionConfig()

config = CLIPSegConfig.from_text_vision_configs(config_text, config_vision)

In [ ]:
print(config)

CLIPSegConfig {
  "_commit_hash": null,
  "conditional_layer": 0,
  "decoder_attention_dropout": 0.0,
  "decoder_hidden_act": "quick_gelu",
  "decoder_intermediate_size": 2048,
  "decoder_num_attention_heads": 4,
  "extract_layers": [
    3,
    6,
    9
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clipseg",
  "projection_dim": 512,
  "reduce_dim": 64,
  "text_config": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 2,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_tok

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPSegModel

processor = AutoProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegModel.from_pretrained("CIDAS/clipseg-rd64-refined")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(
    text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True
)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [ ]:
print(probs)

tensor([[9.9977e-01, 2.3426e-04]], grad_fn=<SoftmaxBackward0>)


In [ ]:
from transformers import AutoTokenizer, CLIPSegModel

tokenizer = AutoTokenizer.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegModel.from_pretrained("CIDAS/clipseg-rd64-refined")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)
print(text_features)

tensor([[ 0.4229, -0.0381,  0.0981,  ...,  0.0605, -0.0413,  0.1337],
        [ 0.3009, -0.1251,  0.1197,  ..., -0.1175,  0.2559,  0.3014]],
       grad_fn=<MmBackward0>)


In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPSegModel

processor = AutoProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegModel.from_pretrained("CIDAS/clipseg-rd64-refined")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, return_tensors="pt")

image_features = model.get_image_features(**inputs)
print(image_features)

tensor([[ 5.0356e-01, -2.6808e-01, -2.6442e-01, -5.1360e-01, -1.2254e-01,
         -2.7451e-01,  1.1046e-01,  2.2918e-01, -2.4197e-01, -5.9845e-01,
          6.6130e-02,  6.5344e-01,  1.3638e-01, -2.1801e-01, -1.7510e-01,
         -5.3189e-01, -7.3737e-02,  4.3500e-01,  5.0601e-01, -1.2943e-01,
          1.3932e-01, -4.4881e-01, -2.4853e-01, -4.6061e-01,  4.5451e-01,
         -2.8225e-01,  6.0254e-02,  4.3775e-02,  2.3510e-01,  1.9383e-02,
          1.9630e-01, -1.1705e-01,  3.4898e-01,  5.4506e-02, -8.1801e-01,
          1.5320e-01,  8.4548e-01,  2.2157e-01, -9.6026e-02, -1.3173e-01,
         -5.0236e-02,  1.7424e-01,  3.2990e-01,  2.5109e-01,  1.3656e-01,
         -1.3673e-01,  1.0319e-02,  6.2809e-01, -2.2358e-01,  2.5092e-01,
         -1.7547e+00, -3.3521e-01,  7.9886e-02, -8.2014e-03,  3.9404e-03,
         -8.2658e-01, -4.8211e-01, -2.9675e-01, -3.2635e-01,  1.1141e-01,
          7.0870e-01, -3.1526e-01, -5.9450e-02, -1.0845e-02,  1.6026e-01,
          7.6576e-02,  1.8985e-01,  1.

In [ ]:
from transformers import AutoTokenizer, CLIPSegTextModel

tokenizer = AutoTokenizer.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegTextModel.from_pretrained("CIDAS/clipseg-rd64-refined")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled (EOS token) states

In [ ]:
print(pooled_output)

tensor([[ 0.0200,  0.0364,  0.5863,  ..., -0.7169,  0.2894, -0.3139],
        [-1.2898, -1.0403,  0.5725,  ..., -0.5004,  0.2878, -0.2958]],
       grad_fn=<IndexBackward0>)


In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPSegVisionModel

processor = AutoProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegVisionModel.from_pretrained("CIDAS/clipseg-rd64-refined")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled CLS states

In [ ]:
print(pooled_output)

tensor([[ 1.2804e-01, -6.9401e-02, -3.5966e-01,  2.1587e-02, -1.6999e-01,
          8.7139e-01,  1.5024e+00, -8.6314e-01, -4.1625e-01, -1.2172e+00,
          1.5193e+00, -1.1043e+00,  8.0011e-01, -5.2581e-01, -5.0075e-01,
         -1.3139e-01, -8.9035e-01, -1.5081e-01,  1.6833e-01,  7.7161e-01,
          1.4833e-01, -1.2581e+00, -3.3301e-01,  1.2403e-01, -2.1614e-02,
          5.2515e-01, -1.0809e-01,  3.0238e-01,  7.8700e-02, -2.2921e-01,
          8.7236e-02,  2.0076e-01, -4.3507e-01,  1.1783e+00,  1.1896e+00,
          8.5576e-01, -2.2777e-01,  1.3100e+00, -1.0232e+00, -7.7997e-01,
          1.2752e+00,  7.6417e-01,  1.6099e+00,  1.1035e+00, -8.4963e-01,
          2.6016e-01,  2.6739e-01, -1.2565e-01, -3.1313e-01,  3.6017e-01,
         -3.4103e-01,  1.0420e+00,  8.6864e-01,  6.5287e-01,  6.9665e-02,
         -2.9488e-01,  1.3064e+00, -1.2646e+00, -2.6310e-01, -5.7449e-01,
          8.6065e-01, -1.7298e-01, -2.0248e+00,  1.2965e+00,  9.0367e-01,
          3.3515e+00, -7.9207e-01,  2.

In [ ]:
from transformers import AutoProcessor, CLIPSegForImageSegmentation
from PIL import Image
import requests

processor = AutoProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
texts = ["a cat", "a remote", "a blanket"]
inputs = processor(text=texts, images=[image] * len(texts), padding=True, return_tensors="pt")

outputs = model(**inputs)

logits = outputs.logits
print(logits.shape)

torch.Size([3, 352, 352])
